## This Notebook will pull in and summarize Robot Scouter JSON Data.

The Notebook exports its results to CSV and Excel files located in a 'LatestResults' directory 
relative to this Jupyter Notebook.

** NO NEED TO RUN THE NEXT CODE SECTION IF YOU HAVE ALREADY INSTALLED THE FOLLOWING PYTHON LIBRARIES **

In [1]:
# NO NEED TO RUN THIS CODE SECTION IF YOU HAVE ALREADY INSTALLED THE FOLLOWING PYTHON LIBRARIES. 
# No problem though if you accidently run it as it will just say the "Requirement already satisfied"

!pip install requests
!pip install pandas

# The following library is currently being used for the Excel writer.  Was planning to use for exporting 
# to an Excel file format. Panda no longer supports exporting directly into an Excel file format.
!pip install openpyxl

In [2]:
import requests
import pandas as pd
from datetime import datetime

In [3]:
# Getting a unique file timestamp
currentTimeStamp = datetime.now()
file_timestamp = currentTimeStamp.strftime("%Y-%m-%d-T%H-%M")

In [4]:
df_scouting = pd.read_json('LatestInputOfScoutData/MATCH_METRICS.json')
display(df_scouting)

,teams
1410,"[{'name': 'Match Metrics 1', 'timestamp': 1647..."
1619,"[{'name': 'Match Metrics 1', 'timestamp': 1647..."
2083,"[{'name': 'Match Metrics 1', 'timestamp': 1647..."
2240,"[{'name': 'Match Metrics 1', 'timestamp': 1647..."
2486,"[{'name': 'Match Metrics 1', 'timestamp': 1647..."
2945,"[{'name': 'Match Metrics 1', 'timestamp': 1647..."
3200,"[{'name': 'Match Azfl Q42', 'timestamp': 16476..."
3374,"[{'name': 'Match Metrics Q62', 'timestamp': 16..."
4068,"[{'name': 'Match Metrics 1', 'timestamp': 1647..."
4293,"[{'name': 'Match Metrics 1', 'timestamp': 1647..."


In [5]:
df_scouting.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13 entries, 1410 to 5690
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   teams   13 non-null     object
dtypes: object(1)
memory usage: 208.0+ bytes


In [6]:
# Making it easier to designate the header / column name to use per metric collected.
d_labels = {'BTaK47cPx6fRaNgEyZUT': 'Scout', 'R0O96f6geNzPSJ9htvbw': 'Scout',
            'aoFxSZtjIt9ivErdykgt': 'Scouted Match', 'audn4HiEfE4mAc4H8sLd': 'AUTO Taxied',
            'YaXQNGcVzGtDxPpy9wgA': 'AUTO Upper Attempts', 'Dl0bTxVPvyB3xmKRaMEe': 'AUTO Upper Made',
            '6QG3gl5E2RyvYvRYnoNr': 'AUTO Lower Attempts', 'advTgK3A354D77txD4YC': 'AUTO Lower Made',
            'ZpWaId8Aq3ryJTpLL3fy': 'TELEOP Upper Attempts', 'bMEWpnglaKMAZd3Ds7fP': 'TELEOP Upper Made',
            'lCxZegc80cFo4FeYJV02': 'TELEOP Lower Attempts', 'L9nU9zioo3j8OgP8ehVa': 'TELEOP Lower Made',
            'qW9pGE83hpB0uFKvZJTo': 'TELEOP Cargo Cycle Time', 'jRFOomtMICa8dRwmVDE9': 'TELEOP Cargo Shooting Comments',
            'jiAEi6v5sNbqvDW1ckAy': 'DEFENSE Rating', 'eMyY331WfizHokzqMMJ1': 'DEFENSE Comments',
            'FHfF2KOElFV9FWnYaFZW': 'END GAME Time To Hang', 'AbcYYzcqQEJwUUhB4sZ3': 'END GAME Climb Rung',
            'lG2IkAch1TRPEHI6f5r3': 'PENALTIES Incurred', 'HLhFaaXhRjiL8Cgaq7wp': 'PENALTIES Comments',
            'NytQ2N41UjzEePOwlQSp': 'TELEOP Speed Rating', 'SceDYFvcXPPW1jlR8zP8': 'RELIABILITY Robot Broke',
            'yPHvlaaDtsi0TGWZMTpJ': 'RELIABILITY Robot Stalled', 'Ig1WZWJkdGycG4CP24OZ': 'RELIABILITY Comments',
            '0zLexNpiJlvudyMtgPyP': 'Additional Comments'
           }       

In [7]:
# Index to use for each row of match metrics that scouts have provided
idx_allmetrics = 0

# A pandas DataFrame to collect all the metrics.  
# Each row is a match and we will identify the team that was scouted in the row. 
df_allmetrics = pd.DataFrame()

# We will collect all the metrics for the current match into a dictionary.
# This dictionary will be added to the end of the df_allmetrics dataframe.
dict_matchmetrics = {}
    
# Let's run through all the scouting match metrics data we have from our scouts
for index_team, row_team in df_scouting.iterrows():
    
    # Let's grab all the matches of metrics that we have for the current team
    df_matches = pd.DataFrame.from_dict(row_team['teams'])
    display(df_matches)
    
    # We'll now iterate through each match's metrics for the current team
    for index_match, row_match in df_matches.iterrows():
        
        # Setting the team number and timestamp for the current match's metrics
        team = index_team
        timestamp = pd.Timestamp(row_match['timestamp'], unit='ms')  
        
        # Adding the team number and timestamp into the dictionary that we will add as a row of metrics
        dict_matchmetrics['Index'] = [idx_allmetrics]        
        dict_matchmetrics['Team'] = [team]
        dict_matchmetrics['Timestamp'] = [timestamp.strftime('%m-%d-%Y')]
        
        # When the JSON file was imported, the metrics were still in a set of key, value, category, name set of fields
        # under the 'metrics' column of the dataframe that was created when we read the JSON file.
        # We will iterate through these key, value type sets of data to parse out what we want from the scouting metrics.
        d_metrics = row_match['metrics']
        
        # Using items() + list comprehension
        # Substring Key match in dictionary
        for key, val in d_metrics.items():
            # print("key: ", key, "Type:", val['type'],"Name:", val['name'], "Category:", val['category'],"Value:", val['value'])
            # print("Label will be:", d_labels[key], 'Index:', i)
            
            if key == 'jiAEi6v5sNbqvDW1ckAy':
                val['value'] = int(val['value'])
            
            # Adding metrics to our dictionary that will be added as a row once we have all of them for the match.
            if val['value'] is None:
                dict_matchmetrics[d_labels[key]] = ['']
            elif len(val['name']) < 6:
                dict_matchmetrics[val['name']] = val['value']              
            elif val['type'] in {"boolean", "number","stopwatch"}:
                dict_matchmetrics[d_labels[key]] = [val['value']]                
            else:
                dict_matchmetrics[d_labels[key]] = [val['value']]                

        print("Dict: ", dict_matchmetrics)
        
        # Let's convert our dictionary of metrics for a match into a DataFrame to be ready to add
        new_matchmetrics = pd.DataFrame(dict_matchmetrics)
        
        # If this is our first row, let's initialize our all metrics DataFrame to this first row.
        if df_allmetrics.empty:
            # If first row let's set it
            df_allmetrics = new_matchmetrics
        else:
            #append new row to the dataframe
            df_allmetrics = pd.concat([df_allmetrics, new_matchmetrics], ignore_index=True)
    
        # Incrementing to be ready to use as an index to each row
        idx_allmetrics = idx_allmetrics + 1

# Let's just see what we have!
display(df_allmetrics)

,name,timestamp,metrics
0,Match Metrics 1,1647394821813,"{'BTaK47cPx6fRaNgEyZUT': {'type': 'text', 'nam..."
1,Match Metrics 1,1647649525898,"{'R0O96f6geNzPSJ9htvbw': {'type': 'text', 'nam..."


Dict:  {'Index': [0], 'Team': [1410], 'Timestamp': ['03-16-2022'], 'Name': 'MN', 'Match': '2022azfl', 'AUTO Taxied': [False], 'AUTO Upper Attempts': [0], 'AUTO Upper Made': [0], 'AUTO Lower Attempts': [0], 'AUTO Lower Made': [0], 'TELEOP Upper Attempts': [0], 'TELEOP Upper Made': [0], 'TELEOP Lower Attempts': [0], 'TELEOP Lower Made': [0], 'TELEOP Cargo Cycle Time': [[]], 'TELEOP Cargo Shooting Comments': [''], 'DEFENSE Rating': [1], 'DEFENSE Comments': [''], 'END GAME Time To Hang': [[]], 'END GAME Climb Rung': ['Unknown '], 'PENALTIES Incurred': [False], 'PENALTIES Comments': [''], 'TELEOP Speed Rating': ['Unknown'], 'RELIABILITY Robot Broke': [False], 'RELIABILITY Robot Stalled': [False], 'RELIABILITY Comments': [''], 'Additional Comments': ['']}
Dict:  {'Index': [1], 'Team': [1410], 'Timestamp': ['03-19-2022'], 'Name': 'Chris', 'Match': 'Qual 42', 'AUTO Taxied': [True], 'AUTO Upper Attempts': [0], 'AUTO Upper Made': [0], 'AUTO Lower Attempts': [1], 'AUTO Lower Made': [1], 'TELEOP U

,name,timestamp,metrics
0,Match Metrics 1,1647384780460,"{'BTaK47cPx6fRaNgEyZUT': {'type': 'text', 'nam..."
1,Match Metrics 1,1647387273889,"{'BTaK47cPx6fRaNgEyZUT': {'type': 'text', 'nam..."
2,Match Metrics 2,1647387477107,"{'BTaK47cPx6fRaNgEyZUT': {'type': 'text', 'nam..."
3,Match Metrics 3,1647387997179,"{'BTaK47cPx6fRaNgEyZUT': {'type': 'text', 'nam..."
4,Match Metrics 2,1647388003628,"{'BTaK47cPx6fRaNgEyZUT': {'type': 'text', 'nam..."
5,Match Metrics F1,1647388006117,"{'BTaK47cPx6fRaNgEyZUT': {'type': 'text', 'nam..."
6,Match Metrics sf2,1647394433656,"{'BTaK47cPx6fRaNgEyZUT': {'type': 'text', 'nam..."
7,Match Metrics sf1-2,1647443862157,"{'BTaK47cPx6fRaNgEyZUT': {'type': 'text', 'nam..."
8,Match Metrics sf1-1,1647444348974,"{'BTaK47cPx6fRaNgEyZUT': {'type': 'text', 'nam..."


Dict:  {'Index': [2], 'Team': [1619], 'Timestamp': ['03-15-2022'], 'Name': 'Noreen', 'Match': 'Okok-f2', 'AUTO Taxied': [True], 'AUTO Upper Attempts': [2], 'AUTO Upper Made': [1], 'AUTO Lower Attempts': [0], 'AUTO Lower Made': [0], 'TELEOP Upper Attempts': [7], 'TELEOP Upper Made': [6], 'TELEOP Lower Attempts': [0], 'TELEOP Lower Made': [0], 'TELEOP Cargo Cycle Time': [[]], 'TELEOP Cargo Shooting Comments': [''], 'DEFENSE Rating': [1], 'DEFENSE Comments': [''], 'END GAME Time To Hang': [[30724]], 'END GAME Climb Rung': ['Traversal '], 'PENALTIES Incurred': [False], 'PENALTIES Comments': [''], 'TELEOP Speed Rating': ['Fast'], 'RELIABILITY Robot Broke': [False], 'RELIABILITY Robot Stalled': [False], 'RELIABILITY Comments': [''], 'Additional Comments': ['Dropped 1 ball']}
Dict:  {'Index': [3], 'Team': [1619], 'Timestamp': ['03-15-2022'], 'Name': 'Chris', 'Match': 'Final 2', 'AUTO Taxied': [True], 'AUTO Upper Attempts': [1], 'AUTO Upper Made': [1], 'AUTO Lower Attempts': [0], 'AUTO Lower M

,name,timestamp,metrics
0,Match Metrics 1,1647650454895,"{'R0O96f6geNzPSJ9htvbw': {'type': 'text', 'nam..."


...

,name,timestamp,metrics
0,Match Metrics Azfl-q41,1647653193189,"{'R0O96f6geNzPSJ9htvbw': {'type': 'text', 'nam..."
1,Match Metrics Azfl-q47,1647791832326,"{'R0O96f6geNzPSJ9htvbw': {'type': 'text', 'nam..."
2,Match Metrics Azfl-q51,1647792127899,"{'R0O96f6geNzPSJ9htvbw': {'type': 'text', 'nam..."
3,Match Metrics Azfl-q57,1647792419783,"{'R0O96f6geNzPSJ9htvbw': {'type': 'text', 'nam..."
4,Match Metrics Azfl-q32,1647792668942,"{'R0O96f6geNzPSJ9htvbw': {'type': 'text', 'nam..."
5,Match Metrics Azfl-q41,1647794481211,"{'R0O96f6geNzPSJ9htvbw': {'type': 'text', 'nam..."
6,Match Metrics Azfl-q1,1647826349380,"{'R0O96f6geNzPSJ9htvbw': {'type': 'text', 'nam..."
7,Match Metrics Azfl-q10,1647826617472,"{'R0O96f6geNzPSJ9htvbw': {'type': 'text', 'nam..."
8,Match Metrics Azfl-q15,1647826813383,"{'R0O96f6geNzPSJ9htvbw': {'type': 'text', 'nam..."
9,Match Metrics Azfl-q19,1647827074842,"{'R0O96f6geNzPSJ9htvbw': {'type': 'text', 'nam..."


Dict:  {'Index': [41], 'Team': [4550], 'Timestamp': ['03-19-2022'], 'Name': 'Chris', 'Match': 'Azfl-q41', 'AUTO Taxied': [True], 'AUTO Upper Attempts': [0], 'AUTO Upper Made': [0], 'AUTO Lower Attempts': [1], 'AUTO Lower Made': [1], 'TELEOP Upper Attempts': [0], 'TELEOP Upper Made': [0], 'TELEOP Lower Attempts': [2], 'TELEOP Lower Made': [2], 'TELEOP Cargo Cycle Time': [[16977]], 'TELEOP Cargo Shooting Comments': [''], 'DEFENSE Rating': [4], 'DEFENSE Comments': ['Got real physical'], 'END GAME Time To Hang': [[6354]], 'END GAME Climb Rung': ['Low'], 'PENALTIES Incurred': [False], 'PENALTIES Comments': [''], 'TELEOP Speed Rating': ['Average '], 'RELIABILITY Robot Broke': [False], 'RELIABILITY Robot Stalled': [False], 'RELIABILITY Comments': ['Reliable for low hub'], 'Additional Comments': ['']}
Dict:  {'Index': [42], 'Team': [4550], 'Timestamp': ['03-20-2022'], 'Name': 'NG', 'Match': 'Azfl-q47', 'AUTO Taxied': [True], 'AUTO Upper Attempts': [0], 'AUTO Upper Made': [0], 'AUTO Lower Attem

,name,timestamp,metrics
0,Match Metrics 1,1647648770344,"{'R0O96f6geNzPSJ9htvbw': {'type': 'text', 'nam..."
1,Match Metrics 2,1647650126981,"{'R0O96f6geNzPSJ9htvbw': {'type': 'text', 'nam..."
2,Match Metrics 3,1647650845638,"{'R0O96f6geNzPSJ9htvbw': {'type': 'text', 'nam..."
3,Match Metrics 4,1647651003701,"{'R0O96f6geNzPSJ9htvbw': {'type': 'text', 'nam..."
4,Match Metrics 5,1647652943885,"{'R0O96f6geNzPSJ9htvbw': {'type': 'text', 'nam..."
5,Match Metrics 6,1647655031980,"{'R0O96f6geNzPSJ9htvbw': {'type': 'text', 'nam..."
6,Match Metrics 7,1647743531839,"{'R0O96f6geNzPSJ9htvbw': {'type': 'text', 'nam..."
7,Match Metrics 8,1647898221917,"{'R0O96f6geNzPSJ9htvbw': {'type': 'text', 'nam..."
8,Match Metrics 9,1647898522088,"{'R0O96f6geNzPSJ9htvbw': {'type': 'text', 'nam..."
9,Match Metrics 10,1647898778282,"{'R0O96f6geNzPSJ9htvbw': {'type': 'text', 'nam..."


Dict:  {'Index': [52], 'Team': [5690], 'Timestamp': ['03-19-2022'], 'Name': 'NG', 'Match': 'Mndu q2m2', 'AUTO Taxied': [True], 'AUTO Upper Attempts': [1], 'AUTO Upper Made': [0], 'AUTO Lower Attempts': [0], 'AUTO Lower Made': [0], 'TELEOP Upper Attempts': [10], 'TELEOP Upper Made': [9], 'TELEOP Lower Attempts': [0], 'TELEOP Lower Made': [0], 'TELEOP Cargo Cycle Time': [[]], 'TELEOP Cargo Shooting Comments': [''], 'DEFENSE Rating': [1], 'DEFENSE Comments': [''], 'END GAME Time To Hang': [[7846]], 'END GAME Climb Rung': ['Mid'], 'PENALTIES Incurred': [False], 'PENALTIES Comments': [''], 'TELEOP Speed Rating': ['Average '], 'RELIABILITY Robot Broke': [False], 'RELIABILITY Robot Stalled': [False], 'RELIABILITY Comments': [''], 'Additional Comments': ['Tippy. Always shot with their front bumper just outside the tarmac']}
Dict:  {'Index': [53], 'Team': [5690], 'Timestamp': ['03-19-2022'], 'Name': 'NG', 'Match': 'Mndu S1m1', 'AUTO Taxied': [True], 'AUTO Upper Attempts': [2], 'AUTO Upper Made'

,Index,Team,Timestamp,Name,Match,AUTO Taxied,AUTO Upper Attempts,AUTO Upper Made,AUTO Lower Attempts,AUTO Lower Made,...,DEFENSE Comments,END GAME Time To Hang,END GAME Climb Rung,PENALTIES Incurred,PENALTIES Comments,TELEOP Speed Rating,RELIABILITY Robot Broke,RELIABILITY Robot Stalled,RELIABILITY Comments,Additional Comments
0,0,1410,03-16-2022,MN,2022azfl,False,0,0,0,0,...,,[],Unknown,False,,Unknown,False,False,,
1,1,1410,03-19-2022,Chris,Qual 42,True,0,0,1,1,...,"Pushed bots around, prevented them from shooting.",[16780],Unknown,False,,Fast,False,False,,Very defensive towards the end of Arizona North
2,2,1619,03-15-2022,Noreen,Okok-f2,True,2,1,0,0,...,,[30724],Traversal,False,,Fast,False,False,,Dropped 1 ball
3,3,1619,03-15-2022,Chris,Final 2,True,1,1,0,0,...,,[9959],Traversal,False,,Fast,False,False,Reliable,
4,4,1619,03-15-2022,Noreen,Okok-f2,True,4,2,0,0,...,,[],Unknown,False,,Unknown,False,False,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,60,5690,03-21-2022,NG,Mndu q13,True,2,2,0,0,...,,[6085],Low,False,,Average,False,False,,
61,61,5690,03-21-2022,NG,Mndu q20,True,1,0,0,0,...,"Played defense for half the game, then switche...",[],None,False,,Average,False,False,First ball in auto didn't go far. Picked up a ...,
62,62,5690,03-21-2022,NG,Mndu q31,True,2,1,0,0,...,,[7353],Mid,False,,Slow,False,False,,They might have trouble getting over the power...
63,63,5690,03-21-2022,NG,Mndu q39,True,2,2,0,0,...,,[6613],Mid,False,,Slow,False,False,,


In [8]:
# Let's see how many matches and what types of columns we have.
df_allmetrics.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65 entries, 0 to 64
Data columns (total 27 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   Index                           65 non-null     int64 
 1   Team                            65 non-null     int64 
 2   Timestamp                       65 non-null     object
 3   Name                            65 non-null     object
 4   Match                           65 non-null     object
 5   AUTO Taxied                     65 non-null     bool  
 6   AUTO Upper Attempts             65 non-null     int64 
 7   AUTO Upper Made                 65 non-null     int64 
 8   AUTO Lower Attempts             65 non-null     int64 
 9   AUTO Lower Made                 65 non-null     int64 
 10  TELEOP Upper Attempts           65 non-null     int64 
 11  TELEOP Upper Made               65 non-null     int64 
 12  TELEOP Lower Attempts           65 non-null     int6

In [9]:
df_allmetrics['Upper Attempts'] = df_allmetrics['AUTO Upper Attempts'] + df_allmetrics['TELEOP Upper Attempts']
df_allmetrics['Upper Made'] = df_allmetrics['AUTO Upper Made'] + df_allmetrics['TELEOP Upper Made']
df_allmetrics['Upper % Made'] = df_allmetrics['Upper Made'] / df_allmetrics['Upper Attempts'] * 100
df_allmetrics['Lower Attempts'] = df_allmetrics['AUTO Lower Attempts'] + df_allmetrics['TELEOP Lower Attempts']
df_allmetrics['Lower Made'] = df_allmetrics['AUTO Lower Made'] + df_allmetrics['TELEOP Lower Made']
df_allmetrics['Lower % Made'] = df_allmetrics['Lower Made'] / df_allmetrics['Lower Attempts'] * 100
display(df_allmetrics)

,Index,Team,Timestamp,Name,Match,AUTO Taxied,AUTO Upper Attempts,AUTO Upper Made,AUTO Lower Attempts,AUTO Lower Made,...,RELIABILITY Robot Broke,RELIABILITY Robot Stalled,RELIABILITY Comments,Additional Comments,Upper Attempts,Upper Made,Upper % Made,Lower Attempts,Lower Made,Lower % Made
0,0,1410,03-16-2022,MN,2022azfl,False,0,0,0,0,...,False,False,,,0,0,NaN,0,0,NaN
1,1,1410,03-19-2022,Chris,Qual 42,True,0,0,1,1,...,False,False,,Very defensive towards the end of Arizona North,0,0,NaN,1,1,100.0
2,2,1619,03-15-2022,Noreen,Okok-f2,True,2,1,0,0,...,False,False,,Dropped 1 ball,9,7,77.777778,0,0,NaN
3,3,1619,03-15-2022,Chris,Final 2,True,1,1,0,0,...,False,False,Reliable,,7,5,71.428571,0,0,NaN
4,4,1619,03-15-2022,Noreen,Okok-f2,True,4,2,0,0,...,False,False,,,4,2,50.000000,0,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60,60,5690,03-21-2022,NG,Mndu q13,True,2,2,0,0,...,False,False,,,13,10,76.923077,0,0,NaN
61,61,5690,03-21-2022,NG,Mndu q20,True,1,0,0,0,...,False,False,First ball in auto didn't go far. Picked up a ...,,7,5,71.428571,1,1,100.0
62,62,5690,03-21-2022,NG,Mndu q31,True,2,1,0,0,...,False,False,,They might have trouble getting over the power...,14,11,78.571429,0,0,NaN
63,63,5690,03-21-2022,NG,Mndu q39,True,2,2,0,0,...,False,False,,,11,9,81.818182,0,0,NaN


In [10]:
df_groupbyTeam = df_allmetrics.groupby('Team').agg(
    max_upper_attempts=pd.NamedAgg(column='Upper Attempts', aggfunc=max),
    min_upper_attempts=pd.NamedAgg(column='Upper Attempts', aggfunc=min),
    mean_upper_attempts=pd.NamedAgg(column='Upper Attempts', aggfunc='mean'),
    max_upper_pctmade=pd.NamedAgg(column='Upper % Made', aggfunc=max),
    min_upper_pctmade=pd.NamedAgg(column='Upper % Made', aggfunc=min),
    mean_upper_pctmade=pd.NamedAgg(column='Upper % Made', aggfunc='mean'),
    max_lower_attempts=pd.NamedAgg(column='Lower Attempts', aggfunc=max),
    min_lower_attempts=pd.NamedAgg(column='Lower Attempts', aggfunc=min),
    mean_lower_attempts=pd.NamedAgg(column='Lower Attempts', aggfunc='mean'),
    max_lower_pctmade=pd.NamedAgg(column='Lower % Made', aggfunc=max),
    min_lower_pctmade=pd.NamedAgg(column='Lower % Made', aggfunc=min),
    mean_lower_pctmade=pd.NamedAgg(column='Lower % Made', aggfunc='mean'),
)

In [11]:
display(df_groupbyTeam)

,max_upper_attempts,min_upper_attempts,mean_upper_attempts,max_upper_pctmade,min_upper_pctmade,mean_upper_pctmade,max_lower_attempts,min_lower_attempts,mean_lower_attempts,max_lower_pctmade,min_lower_pctmade,mean_lower_pctmade
Team,,,,,,,,,,,,
1410,0,0,0.000000,NaN,NaN,NaN,1,0,0.500000,100.000000,100.000000,100.000000
1619,17,4,12.333333,85.714286,50.000000,68.329184,0,0,0.000000,NaN,NaN,NaN
2083,0,0,0.000000,NaN,NaN,NaN,2,2,2.000000,50.000000,50.000000,50.000000
2240,2,2,2.000000,0.000000,0.000000,0.000000,6,6,6.000000,83.333333,83.333333,83.333333
2486,8,8,8.000000,62.500000,62.500000,62.500000,1,1,1.000000,0.000000,0.000000,0.000000
2945,8,1,4.000000,100.000000,42.857143,73.809524,0,0,0.000000,NaN,NaN,NaN
3200,5,0,2.500000,60.000000,60.000000,60.000000,0,0,0.000000,NaN,NaN,NaN
3374,17,3,13.000000,100.000000,58.823529,82.606952,0,0,0.000000,NaN,NaN,NaN
4068,2,2,2.000000,50.000000,50.000000,50.000000,20,20,20.000000,100.000000,100.000000,100.000000


In [12]:
# Let's remove all the '_' underscores from our names
df_groupbyTeam.columns = df_groupbyTeam.columns.str.replace("_", " ")
# Let's remove all the '_' underscores from our names
df_groupbyTeam.columns = df_groupbyTeam.columns.str.replace("pct", "% ")

df_groupbyTeam.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 13 entries, 1410 to 5690
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   max upper attempts   13 non-null     int64  
 1   min upper attempts   13 non-null     int64  
 2   mean upper attempts  13 non-null     float64
 3   max upper % made     8 non-null      float64
 4   min upper % made     8 non-null      float64
 5   mean upper % made    8 non-null      float64
 6   max lower attempts   13 non-null     int64  
 7   min lower attempts   13 non-null     int64  
 8   mean lower attempts  13 non-null     float64
 9   max lower % made     9 non-null      float64
 10  min lower % made     9 non-null      float64
 11  mean lower % made    9 non-null      float64
dtypes: float64(8), int64(4)
memory usage: 1.3 KB


## Output the full list of Match Metrics we have from the Scouts

In [13]:
#  All match metrics exporting to a CSV file to be used by other scripts or to look at AS-IS
df_allmetrics[['Team','Timestamp','Name','Match','Upper Attempts','Upper % Made','AUTO Upper Attempts','AUTO Upper Made','TELEOP Upper Attempts','TELEOP Upper Made','Lower Attempts','Lower % Made','AUTO Lower Attempts','AUTO Lower Made','TELEOP Lower Attempts','TELEOP Lower Made','TELEOP Cargo Shooting Comments','AUTO Taxied','END GAME Climb Rung','TELEOP Speed Rating','END GAME Time To Hang','TELEOP Cargo Cycle Time','DEFENSE Rating','DEFENSE Comments','PENALTIES Incurred','PENALTIES Comments','RELIABILITY Robot Broke','RELIABILITY Robot Stalled','RELIABILITY Comments','Additional Comments']].to_csv("LatestResults/AllMatchMetrics.csv", index=True, float_format="%.1f")

In [14]:
#  The summary[max, min, mean] of match metrics by team exporting to a CSV file to be used by other scripts or to look at AS-IS
df_groupbyTeam[['max upper attempts','min upper attempts','mean upper attempts','max upper % made','min upper % made','mean upper % made','max lower attempts','min lower attempts','mean lower attempts','max lower % made','min lower % made','mean lower % made']].to_csv("LatestResults/MaxMinMeanOfTeams.csv", index=True, float_format="%.1f")

In [15]:
# Attempting to write to Excel to look at AS-IS 
filename_Excel = "LatestResults/OurScouting.xlsx"
with pd.ExcelWriter(filename_Excel, datetime_format='mmm d yyyy hh:mm:ss', date_format='hh:mm:ss.000') as writer:
    df_groupbyTeam[['max upper attempts','min upper attempts','mean upper attempts','max upper % made','min upper % made','mean upper % made','max lower attempts','min lower attempts','mean lower attempts','max lower % made','min lower % made','mean lower % made']].to_excel(writer, sheet_name="MaxMinMean of Teams", index=True, float_format="%.1f")
    df_allmetrics[['Team','Timestamp','Name','Match','Upper Attempts','Upper % Made','Lower Attempts','Lower % Made','AUTO Upper Attempts','AUTO Upper Made','AUTO Lower Attempts','AUTO Lower Made','TELEOP Upper Attempts','TELEOP Upper Made','TELEOP Lower Attempts','TELEOP Lower Made','TELEOP Cargo Shooting Comments','AUTO Taxied','END GAME Climb Rung','TELEOP Speed Rating','TELEOP Cargo Cycle Time','END GAME Time To Hang']].to_excel(writer, sheet_name="Shooting Offensive", index=False, float_format="%.1f")
    df_allmetrics[['Team','Timestamp','Name','Match','AUTO Taxied','END GAME Climb Rung','TELEOP Speed Rating','TELEOP Cargo Cycle Time','END GAME Time To Hang']].to_excel(writer, sheet_name="Other Offensive", index=False, float_format="%.1f")
    df_allmetrics[['Team','Timestamp','Name','Match','DEFENSE Rating','DEFENSE Comments','PENALTIES Incurred','PENALTIES Comments']].to_excel(writer, sheet_name="Defense", index=False, float_format="%.1f")
    df_allmetrics[['Team','Timestamp','Name','Match','RELIABILITY Robot Broke','RELIABILITY Robot Stalled','RELIABILITY Comments','Additional Comments']].to_excel(writer, sheet_name="Reliability", index=False, float_format="%.1f")
